In [ ]:
# Master Benchmarking for Deep Learning Models

In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]="0" # Nvidia Quadro GV100
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # Nvidia Quadro M2000

#Adding Path to various Modules
sys.path.append("../core")
sys.path.append("../visualization")
sys.path.append("../utilities")
sys.path.append("../datasets")
sys.path.append("../trained_models")
sys.path.append("../config")

In [2]:
#Importing Required Modules
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()

#Importing Config files
import assembly_config as config
import model_config as cftrain
import hybrid_utils as hy_util
#Importing required modules from the package
from measurement_system import HexagonWlsScanner
from assembly_system import VRMSimulationModel
from wls400a_system import GetInferenceData
from data_import import GetTrainData
from encode_decode_model import Encode_Decode_Model
from training_viz import TrainViz
from metrics_eval import MetricsEval
from keras_lr_multiplier import LRMultiplier

Using TensorFlow backend.


In [3]:
print('Parsing from Assembly Config File....')

data_type=config.assembly_system['data_type']
application=config.assembly_system['application']
part_type=config.assembly_system['part_type']
part_name=config.assembly_system['part_name']
data_format=config.assembly_system['data_format']
assembly_type=config.assembly_system['assembly_type']
assembly_kccs=config.assembly_system['assembly_kccs']
assembly_kpis=config.assembly_system['assembly_kpis']
voxel_dim=config.assembly_system['voxel_dim']
point_dim=config.assembly_system['point_dim']
voxel_channels=config.assembly_system['voxel_channels']
noise_type=config.assembly_system['noise_type']
mapping_index=config.assembly_system['mapping_index']

system_noise=config.assembly_system['system_noise']
aritifical_noise=config.assembly_system['aritifical_noise']
data_folder=config.assembly_system['data_folder']
kcc_folder=config.assembly_system['kcc_folder']
kcc_files=config.assembly_system['kcc_files']
test_kcc_files=config.assembly_system['test_kcc_files']

Parsing from Assembly Config File....


In [4]:
#added for hybrid model
categorical_kccs=config.assembly_system['categorical_kccs']

print('Parsing from Training Config File')

model_type=cftrain.model_parameters['model_type']
output_type=cftrain.model_parameters['output_type']
batch_size=cftrain.model_parameters['batch_size']
epocs=cftrain.model_parameters['epocs']
split_ratio=cftrain.model_parameters['split_ratio']
optimizer=cftrain.model_parameters['optimizer']
loss_func=cftrain.model_parameters['loss_func']
regularizer_coeff=cftrain.model_parameters['regularizer_coeff']
activate_tensorboard=cftrain.model_parameters['activate_tensorboard']

Parsing from Training Config File


In [50]:
print('Creating file Structure....')

bn_model_name='fcnn_bn'

folder_name=part_type
train_path='../trained_models/'+part_type
pathlib.Path(train_path).mkdir(parents=True, exist_ok=True) 

train_path=train_path+'/'+bn_model_name
pathlib.Path(train_path).mkdir(parents=True, exist_ok=True) 

model_path=train_path+'/models'
pathlib.Path(model_path).mkdir(parents=True, exist_ok=True)

logs_path=train_path+'/logs'
pathlib.Path(logs_path).mkdir(parents=True, exist_ok=True)

plots_path=train_path+'/plots'
pathlib.Path(plots_path).mkdir(parents=True, exist_ok=True)


Creating file Structure....


In [6]:
#Objects of Measurement System, Assembly System, Get Inference Data
print('Initializing the Assembly System and Measurement System....')
measurement_system=HexagonWlsScanner(data_type,application,system_noise,part_type,data_format)
vrm_system=VRMSimulationModel(assembly_type,assembly_kccs,assembly_kpis,part_name,part_type,voxel_dim,voxel_channels,point_dim,aritifical_noise)
get_data=GetTrainData()


kcc_sublist=cftrain.encode_decode_params['kcc_sublist']
output_heads=cftrain.encode_decode_params['output_heads']
encode_decode_multi_output_construct=config.encode_decode_multi_output_construct

if(output_heads==len(encode_decode_multi_output_construct)):
	print("Valid Output Stages and heads")
else:
	print("Inconsistent model setting")

print("KCC sub-listing: ",kcc_sublist)

#Check for KCC sub-listing
if(kcc_sublist!=0):
	output_dimension=len(kcc_sublist)
else:
	output_dimension=assembly_kccs

print("Process Parameter Dimension: ",output_dimension)

input_size=(voxel_dim,voxel_dim,voxel_dim,voxel_channels)

model_depth=cftrain.encode_decode_params['model_depth']
inital_filter_dim=cftrain.encode_decode_params['inital_filter_dim']


Initializing the Assembly System and Measurement System....
Valid Output Stages and heads
KCC sub-listing:  0
Process Parameter Dimension:  157


In [7]:
#importing file names for model input
input_file_names_x=config.encode_decode_construct['input_data_files_x']
input_file_names_y=config.encode_decode_construct['input_data_files_y']
input_file_names_z=config.encode_decode_construct['input_data_files_z']

input_dataset=[]
input_dataset.append(get_data.data_import(input_file_names_x,data_folder))
input_dataset.append(get_data.data_import(input_file_names_y,data_folder))
input_dataset.append(get_data.data_import(input_file_names_z,data_folder))

kcc_dataset=get_data.data_import(kcc_files,kcc_folder)

if(kcc_sublist!=0):
	print("Sub-setting Process Parameters: ",kcc_sublist)
	kcc_dataset=kcc_dataset.iloc[:,kcc_sublist]
	test_kcc_dataset=test_kcc_dataset[:,kcc_sublist]
else:
	print("Using all Process Parameter")

Using all Process Parameter


In [8]:
#Pre-processing to point cloud data
point_index=get_data.load_mapping_index(mapping_index)
input_conv_data, kcc_subset_dump,kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,input_dataset,point_index,kcc_dataset)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.72it/s]

Number of not convergent solutions:  0


In [9]:
y_shape_error_list=[]

for encode_decode_construct in encode_decode_multi_output_construct:
	#importing file names for model output
	print("Importing output data for stage: ",encode_decode_construct)

	output_file_names_x=encode_decode_construct['output_data_files_x']
	output_file_names_y=encode_decode_construct['output_data_files_y']
	output_file_names_z=encode_decode_construct['output_data_files_z']

	output_dataset=[]
	output_dataset.append(get_data.data_import(output_file_names_x,data_folder))
	output_dataset.append(get_data.data_import(output_file_names_y,data_folder))
	output_dataset.append(get_data.data_import(output_file_names_z,data_folder))

	output_conv_data, kcc_subset_dump,kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,output_dataset,point_index,kcc_dataset)

	y_shape_error_list.append(output_conv_data)

shape_error=np.concatenate(y_shape_error_list, axis=4)
kcc_regression,kcc_classification=hy_util.split_kcc(kcc_subset_dump)

Importing output data for stage:  {'stage_id': 2, 'output_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_data_files_z': ['DZ_crossmember_test1_3.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_3.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.80it/s]


Number of not convergent solutions:  0
Importing output data for stage:  {'stage_id': 6, 'output_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_data_files_z': ['DZ_crossmember_test1_7.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_7.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.71it/s]


Number of not convergent solutions:  0


In [36]:
# Prepare Benchmarking Input Output
# 1 Only Regression
# 2 Only Classfication
# 3 Only Shape Error
# 4 Regression + Classification
# 5 Regression + OSE
# 6 Regression + Classification
# 7 Regression + Classification + OSE
#Select Option to execute
option_num=3

from bm_selector import get_bm_io

y_out_list=get_bm_io(option_num,kcc_regression,kcc_classification,shape_error)


In [42]:
# Build Model using Keras Tuner

#Get Base Model
#Reloading to change cache
import importlib
import dl_models
importlib.reload(dl_models)

dl_model_obj2=dl_models.DL_BM_Arch(output_dimension)


In [43]:
#Import Benchmarking Model
model=dl_model_obj2.fcnn()

3D FCNN model successfully compiled
Model: "FCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 64, 64, 64, 3)]   0         
_________________________________________________________________
Conv0_1 (Conv3D)             (None, 64, 64, 64, 32)    2624      
_________________________________________________________________
Act0_1 (Activation)          (None, 64, 64, 64, 32)    0         
_________________________________________________________________
MaxPooling0_1 (MaxPooling3D) (None, 32, 32, 32, 32)    0         
_________________________________________________________________
Conv1_1 (Conv3D)             (None, 32, 32, 32, 64)    55360     
_________________________________________________________________
Act1_1 (Activation)          (None, 32, 32, 32, 64)    0         
_________________________________________________________________
MaxPooling1_1 (MaxPooling3

In [51]:
##Train Benchmarking Model
from tensorflow.keras import backend as K
weight_path=model_path+'/fcnn_0'
checkpointer = tf.keras.callbacks.ModelCheckpoint(weight_path, verbose=1, save_best_only='val_loss',save_weights_only=True)
history=model.fit(x=input_conv_data,y=y_out_list, validation_split=0.3, epochs=epocs, batch_size=batch_size,callbacks=[checkpointer])

Train on 350 samples, validate on 150 samples
Epoch 1/3
336/350 [===========================>..] - ETA: 0s - loss: 0.0044 - mean_absolute_error: 0.0070
Epoch 00001: val_loss improved from inf to 0.00424, saving model to ../trained_models/cross_member_assembly/fcnn_bn/models/fcnn_0
350/350 [==============================] - 22s 62ms/sample - loss: 0.0044 - mean_absolute_error: 0.0070 - val_loss: 0.0042 - val_mean_absolute_error: 0.0068
Epoch 2/3
336/350 [===========================>..] - ETA: 0s - loss: 0.0044 - mean_absolute_error: 0.0067
Epoch 00002: val_loss improved from 0.00424 to 0.00422, saving model to ../trained_models/cross_member_assembly/fcnn_bn/models/fcnn_0
350/350 [==============================] - 21s 61ms/sample - loss: 0.0044 - mean_absolute_error: 0.0067 - val_loss: 0.0042 - val_mean_absolute_error: 0.0061
Epoch 3/3
336/350 [===========================>..] - ETA: 0s - loss: 0.0043 - mean_absolute_error: 0.0065
Epoch 00003: val_loss improved from 0.00422 to 0.00405, sa

In [46]:
# Import Test Dataset
test_input_file_names_x=config.encode_decode_construct['input_test_data_files_x']
test_input_file_names_y=config.encode_decode_construct['input_test_data_files_y']
test_input_file_names_z=config.encode_decode_construct['input_test_data_files_z']
test_input_dataset=[]
test_input_dataset.append(get_data.data_import(test_input_file_names_x,data_folder))
test_input_dataset.append(get_data.data_import(test_input_file_names_y,data_folder))
test_input_dataset.append(get_data.data_import(test_input_file_names_z,data_folder))
test_kcc_dataset=get_data.data_import(test_kcc_files,kcc_folder)
test_input_conv_data, test_kcc_subset_dump,test_kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,test_input_dataset,point_index,test_kcc_dataset)

kcc_regression_test,kcc_classification_test=hy_util.split_kcc(test_kcc_subset_dump)
Y_out_test_list=[]
Y_out_test_list.append(kcc_regression_test)
Y_out_test_list.append(kcc_classification_test)

y_shape_error_test_list=[]

for encode_decode_construct in encode_decode_multi_output_construct:
	#importing file names for model output
	print("Importing output data for stage: ",encode_decode_construct)


	test_output_file_names_x=encode_decode_construct['output_test_data_files_x']
	test_output_file_names_y=encode_decode_construct['output_test_data_files_y']
	test_output_file_names_z=encode_decode_construct['output_test_data_files_z']

	test_output_dataset=[]
	test_output_dataset.append(get_data.data_import(test_output_file_names_x,data_folder))
	test_output_dataset.append(get_data.data_import(test_output_file_names_y,data_folder))
	test_output_dataset.append(get_data.data_import(test_output_file_names_z,data_folder))

	test_output_conv_data, test_kcc_subset_dump,test_kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,test_output_dataset,point_index,test_kcc_dataset)

	y_shape_error_test_list.append(test_output_conv_data)

shape_error_test=np.concatenate(y_shape_error_test_list, axis=4)

Y_out_test_list.append(shape_error_test)


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.85it/s]


Number of not convergent solutions:  0
Splitting Contionous and Categorical KCCs
Valid Split
Importing output data for stage:  {'stage_id': 2, 'output_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_data_files_z': ['DZ_crossmember_test1_3.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_3.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.82it/s]


Number of not convergent solutions:  0
Importing output data for stage:  {'stage_id': 6, 'output_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_data_files_z': ['DZ_crossmember_test1_7.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_7.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:42<00:00, 11.83it/s]


Number of not convergent solutions:  0


In [53]:
# Import Weights for the best model
model.load_weights(weight_path)
y_pred_list=model.predict(test_input_conv_data)

In [54]:
# Check for Consistent Output
len(y_pred_list)

500

In [69]:
# Get Test Metrics based on model output
from metrics_eval import MetricsEval
metrics_eval=MetricsEval();

if(option_num==1):
    eval_metrics_reg,accuracy_metrics_df_reg=metrics_eval.metrics_eval_base(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_reg.to_csv(logs_path+'/metrics_test_regression.csv')

if(option_num==2):
    eval_metrics_cla,accuracy_metrics_df_cla=metrics_eval.metrics_eval_classification(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_cla.to_csv(logs_path+'/metrics_test_classification.csv')

if(option_num==3):
    eval_metrics_cop_list=[]
    accuracy_metrics_df_cop_list=[]
    t=0
    index=0
    for i in range(output_heads):
        y_cop_pred=y_pred_list[:,:,:,:,t:t+3]
        y_cop_test=Y_out_test_list[2][:,:,:,:,t:t+3]
        y_cop_actual=y_cop_test
        y_cop_pred_vector=np.reshape(y_cop_pred,(y_cop_pred.shape[0],-1))
        y_cop_test_vector=np.reshape(y_cop_test,(y_cop_test.shape[0],-1))
        y_cop_pred_vector=y_cop_pred_vector.T
        y_cop_test_vector=y_cop_test_vector.T
        print(y_cop_pred_vector.shape)
        eval_metrics_cop,accuracy_metrics_df_cop=metrics_eval.metrics_eval_cop(y_cop_pred_vector,y_cop_test_vector,logs_path)
        eval_metrics_cop_list.append(eval_metrics_cop)
        accuracy_metrics_df_cop_list.append(accuracy_metrics_df_cop)
        accuracy_metrics_df_cop.to_csv(logs_path+'/metrics_test_cop_'+str(index)+'.csv')
        print("The Model Segmentation Validation Metrics are ")
        print(accuracy_metrics_df_cop.mean())
        accuracy_metrics_df_cop.mean().to_csv(logs_path+'/metrics_test_cop_summary_'+str(index)+'.csv')     
        t=t+3
        index=index+1  

if(option_num==4):
    eval_metrics_reg,accuracy_metrics_df_reg=metrics_eval.metrics_eval_base(y_pred_list[0],Y_out_test_list[0],logs_path)
    eval_metrics_cla,accuracy_metrics_df_cla=metrics_eval.metrics_eval_classification(y_pred_list[1],Y_out_test_list[1],logs_path)
    accuracy_metrics_df_reg.to_csv(logs_path+'/metrics_test_regression.csv')
    accuracy_metrics_df_cla.to_csv(logs_path+'/metrics_test_classification.csv')


if(option_num==5):
    eval_metrics_reg,accuracy_metrics_df_reg=metrics_eval.metrics_eval_base(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_reg.to_csv(logs_path+'/metrics_test_regression.csv')
    eval_metrics_cop_list=[]
    accuracy_metrics_df_cop_list=[]
    t=0
    index=0
    for i in range(output_heads):
        y_cop_pred=y_pred_list[:,:,:,:,t:t+3]
        y_cop_test=Y_out_test_list[2][:,:,:,:,t:t+3]
        y_cop_actual=y_cop_test
        y_cop_pred_vector=np.reshape(y_cop_pred,(y_cop_pred.shape[0],-1))
        y_cop_test_vector=np.reshape(y_cop_test,(y_cop_test.shape[0],-1))
        y_cop_pred_vector=y_cop_pred_vector.T
        y_cop_test_vector=y_cop_test_vector.T
        print(y_cop_pred_vector.shape)
        eval_metrics_cop,accuracy_metrics_df_cop=metrics_eval.metrics_eval_cop(y_cop_pred_vector,y_cop_test_vector,logs_path)
        eval_metrics_cop_list.append(eval_metrics_cop)
        accuracy_metrics_df_cop_list.append(accuracy_metrics_df_cop)
        accuracy_metrics_df_cop.to_csv(logs_path+'/metrics_test_cop_'+str(index)+'.csv')
        print("The Model Segmentation Validation Metrics are ")
        print(accuracy_metrics_df_cop.mean())
        accuracy_metrics_df_cop.mean().to_csv(logs_path+'/metrics_test_cop_summary_'+str(index)+'.csv')     
        t=t+3
        index=index+1 
    

if(option_num==6):
    eval_metrics_cla,accuracy_metrics_df_cla=metrics_eval.metrics_eval_classification(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_cla.to_csv(logs_path+'/metrics_test_classification.csv')
    eval_metrics_cop_list=[]
    accuracy_metrics_df_cop_list=[]
    t=0
    index=0
    for i in range(output_heads):
        y_cop_pred=y_pred_list[:,:,:,:,t:t+3]
        y_cop_test=Y_out_test_list[2][:,:,:,:,t:t+3]
        y_cop_actual=y_cop_test
        y_cop_pred_vector=np.reshape(y_cop_pred,(y_cop_pred.shape[0],-1))
        y_cop_test_vector=np.reshape(y_cop_test,(y_cop_test.shape[0],-1))
        y_cop_pred_vector=y_cop_pred_vector.T
        y_cop_test_vector=y_cop_test_vector.T
        print(y_cop_pred_vector.shape)
        eval_metrics_cop,accuracy_metrics_df_cop=metrics_eval.metrics_eval_cop(y_cop_pred_vector,y_cop_test_vector,logs_path)
        eval_metrics_cop_list.append(eval_metrics_cop)
        accuracy_metrics_df_cop_list.append(accuracy_metrics_df_cop)
        accuracy_metrics_df_cop.to_csv(logs_path+'/metrics_test_cop_'+str(index)+'.csv')
        print("The Model Segmentation Validation Metrics are ")
        print(accuracy_metrics_df_cop.mean())
        accuracy_metrics_df_cop.mean().to_csv(logs_path+'/metrics_test_cop_summary_'+str(index)+'.csv')     
        t=t+3
        index=index+1 

if(option_num==7):
    eval_metrics_reg,accuracy_metrics_df_reg=metrics_eval.metrics_eval_base(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_reg.to_csv(logs_path+'/metrics_test_regression.csv')
    eval_metrics_cla,accuracy_metrics_df_cla=metrics_eval.metrics_eval_classification(y_pred_list,Y_out_test_list[0],logs_path)
    accuracy_metrics_df_cla.to_csv(logs_path+'/metrics_test_classification.csv')
    eval_metrics_cop_list=[]
    accuracy_metrics_df_cop_list=[]
    t=0
    index=0
    for i in range(output_heads):
        y_cop_pred=y_pred_list[:,:,:,:,t:t+3]
        y_cop_test=Y_out_test_list[2][:,:,:,:,t:t+3]
        y_cop_actual=y_cop_test
        y_cop_pred_vector=np.reshape(y_cop_pred,(y_cop_pred.shape[0],-1))
        y_cop_test_vector=np.reshape(y_cop_test,(y_cop_test.shape[0],-1))
        y_cop_pred_vector=y_cop_pred_vector.T
        y_cop_test_vector=y_cop_test_vector.T
        print(y_cop_pred_vector.shape)
        eval_metrics_cop,accuracy_metrics_df_cop=metrics_eval.metrics_eval_cop(y_cop_pred_vector,y_cop_test_vector,logs_path)
        eval_metrics_cop_list.append(eval_metrics_cop)
        accuracy_metrics_df_cop_list.append(accuracy_metrics_df_cop)
        accuracy_metrics_df_cop.to_csv(logs_path+'/metrics_test_cop_'+str(index)+'.csv')
        print("The Model Segmentation Validation Metrics are ")
        print(accuracy_metrics_df_cop.mean())
        accuracy_metrics_df_cop.mean().to_csv(logs_path+'/metrics_test_cop_summary_'+str(index)+'.csv')     
        t=t+3
        index=index+1 
    

(786432, 500)


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 85.49it/s]


The Model Segmentation Validation Metrics are 
MAE              0.002010
MSE              0.001285
RMSE             0.032435
R2            -394.871946
R2_Adjusted     -0.171528
dtype: float64
(786432, 500)


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:06<00:00, 82.25it/s]

The Model Segmentation Validation Metrics are 
MAE             0.010409
MSE             0.007118
RMSE            0.081886
R2            -34.017012
R2_Adjusted     0.034769
dtype: float64


In [72]:
# Plotting and saving additional files for VRM Plotting
from point_cloud_construction import GetPointCloud
import voxel_config as vc

get_point_cloud=GetPointCloud()

cop_file_name=vc.voxel_parameters['nominal_cop_filename']
cop_file_path='../resources/nominal_cop_files/'+cop_file_name

#Read cop from csv file
print('Importing Nominal COP')
nominal_cop=vrm_system.get_nominal_cop(cop_file_path)
deploy_path=logs_path

t=0
index=0

for i in range(output_heads):
    y_cop_pred=y_pred_list[:,:,:,:,t:t+3]
    y_cop_test=Y_out_test_list[2][:,:,:,:,t:t+3] #Change List ID based on output type
    y_cop_actual=y_cop_test

    #Saving For Matlab Plotting
    dev_pred_matlab_plot_x=np.zeros((len(y_cop_pred),point_dim))
    dev_pred_matlab_plot_y=np.zeros((len(y_cop_pred),point_dim))
    dev_pred_matlab_plot_z=np.zeros((len(y_cop_pred),point_dim))

    dev_actual_matlab_plot_x=np.zeros((len(y_cop_pred),point_dim))
    dev_actual_matlab_plot_y=np.zeros((len(y_cop_pred),point_dim))
    dev_actual_matlab_plot_z=np.zeros((len(y_cop_pred),point_dim))

    # Saving for Matlab plotting
    print("Saving Files for VRM Plotting...")

    from tqdm import tqdm
    for i in tqdm(range(len(y_cop_pred))):
        actual_dev=get_point_cloud.getcopdev(y_cop_actual[i,:,:,:,:],point_index,nominal_cop)
        pred_dev=get_point_cloud.getcopdev(y_cop_pred[i,:,:,:,:],point_index,nominal_cop)
        dev_pred_matlab_plot_x[i,:]=pred_dev[:,0]
        dev_pred_matlab_plot_y[i,:]=pred_dev[:,1]
        dev_pred_matlab_plot_z[i,:]=pred_dev[:,2]
        dev_actual_matlab_plot_x[i,:]=actual_dev[:,0]
        dev_actual_matlab_plot_y[i,:]=actual_dev[:,1]
        dev_actual_matlab_plot_z[i,:]=actual_dev[:,2]

    np.savetxt((logs_path+'/DX_pred_'+str(index)+'.csv'),dev_pred_matlab_plot_x, delimiter=",")
    np.savetxt((logs_path+'/DY_pred_'+str(index)+'.csv'),dev_pred_matlab_plot_y, delimiter=",")
    np.savetxt((logs_path+'/DZ_pred_'+str(index)+'.csv'),dev_pred_matlab_plot_z, delimiter=",")
    
    np.savetxt((logs_path+'/DX_actual_'+str(index)+'.csv'),dev_actual_matlab_plot_x, delimiter=",")
    np.savetxt((logs_path+'/DY_actual_'+str(index)+'.csv'),dev_actual_matlab_plot_y, delimiter=",")
    np.savetxt((logs_path+'/DZ_actual_'+str(index)+'.csv'),dev_actual_matlab_plot_z, delimiter=",")	
    t=t+3
    index=index+1

  0%|▎                                                                                 | 2/500 [00:00<00:26, 18.60it/s]

Importing Nominal COP
Saving Files for VRM Plotting...


  1%|▍                                                                                 | 3/500 [00:00<00:24, 20.70it/s]

Saving Files for VRM Plotting...


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:23<00:00, 20.97it/s]
